In [1]:
import csv
import docx
from tqdm import tqdm
import utils

In [2]:
# test section
utils.get_sec_num('23.1 alwijdli')

'23.1'

In [4]:
# prepare replacing data in sections
replace = {}
replace_table = {}
with open('dell.csv') as file:
    data = csv.reader(file,delimiter=',')
    count = 0
    for row in data:  
        count += 1
        if count <= 3:
            continue
        
        sec_num = utils.get_sec_num(row[1])       
        
        if sec_num is not None:
            # tmp solution: self-increment for a new added-in section
            tmp_s = [int(x) for x in sec_num.split('.')]
            tmp_s[0] += 1
            
            sec_num = ".".join([str(x) for x in tmp_s])
            if sec_num not in replace.keys():
                replace[sec_num] = []
            replace[sec_num].append((row[2],row[3],row[0]))  # sec_num : (before, after, index)
        else:
            replace_table[row[2]] = (row[3],row[0])  # before : (after, index)


In [6]:
in_path = '/home/gray/desktop/7.15.docx'
doc = docx.Document(in_path)

In [50]:
found = {}  # 格式 "before : [counter, index]"
indexes = []  # 审核通过的（只出现了一次的）

# search in paragraphs
now_sec = ''
for para in tqdm(list(doc.paragraphs)):
    # update sec_num
    sec_num = utils.get_sec_num(para.text)
    if sec_num is not None:
        now_sec = sec_num
#         print(now_sec)
    
    if now_sec == '':
        continue
    
    if now_sec not in replace.keys():
        continue
    
    for a,b,index in replace[now_sec]:
#         for run in para.runs:
#             if a != '' and a in run.text:
#                 if a not in found.keys():
#                     found[a] = 0
#                 found[a] += 1

        if a != '' and a in para.text:
            '''
            do replace(a,b)
            '''
            if a not in found.keys():
                found[a] = [0,b,index]
            
            found[a][0] += 1

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12098/12098 [00:00<00:00, 58632.45it/s]

200


In [51]:
found_table = {}  # 格式 "before : [counter, afer, index]"
for table_index in tqdm(range(len(list(doc.tables)))[10:]):
    table = doc.tables[table_index]
    for row in table.rows:
        for cell in row.cells:
            for para in cell.paragraphs:
                for a,b in replace_table.items():
                    if a!='' and a in para.text:
                        if a not in found_table.keys():
                            found_table[a] = [0]+list(b)
                        found_table[a][0] += 1
# print(found_table)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 713/713 [01:03<00:00, 11.16it/s]

124


In [61]:
print(list(found_table.items())[0])
print(list(found.items())[0])

('Job Control', [11, '作业控制', '6'])
('中心类', [1, '核心类', '2'])


In [68]:
indexes = []
for k,v in found.items():
    if v[0] ==  1:
        indexes.append(v[2])

for k,v in found_table.items():
    if v[0] <= 1:
        indexes.append(v[2])
        
print(len(indexes))
print(indexes[:10])

324
['2', '3', '9', '10', '11', '13', '14', '15', '16', '17']


In [ ]:
for k,v in found_table.items():
    if v[0] == 1:
        indexes.append(v[2])

In [65]:
data = []
with open('dell.csv', 'r') as file:
    reader = csv.reader(file,delimiter=',')
    for row in reader:
        if row[0] in indexes:
            row.append('python')
        data.append(row)

In [66]:
with open('/home/gray/desktop/dell_n.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerows(data)